# Partitioned Custom Time Series Model - Statsmodels Exponential Smoothing

This notebook shows how to partition a time series model. It uses synthetic data and statsmodels exponetial smoothing. The notebooks shows how to test it locally and then run the model in a distributed fashion in Snowflake. I have also made it so you can push the datasets into a Snowflake table for running the inference from the Snowflake model registry.

In [ ]:
import json
from snowflake.snowpark import Session
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend

with open('../../creds.json') as f:
    data = json.load(f)
    USERNAME = data['user']
    SF_ACCOUNT = data['account']
    SF_WH = data['warehouse']
    passphrase = data['passphrase']

# Read the private key from the .p8 file
with open('../../rsa_key.p8', 'rb') as key_file:
    private_key = key_file.read()

# If the private key is encrypted, load it with a passphrase
# Replace 'your_key_passphrase' with your actual passphrase if needed
private_key_obj = serialization.load_pem_private_key(
    private_key,
    password=passphrase.encode() if passphrase else None,
    backend=default_backend()
)

# Define connection parameters including the private key
CONNECTION_PARAMETERS = {
    'user': USERNAME,
    'account': SF_ACCOUNT,
    'private_key': private_key_obj,
    'warehouse': SF_WH,
}

# Create a session with the specified connection parameters
session = Session.builder.configs(CONNECTION_PARAMETERS).create()

from snowflake.core.warehouse import Warehouse
from snowflake.core import Root
root = Root(session)
from snowflake.snowpark.functions import col 
from time import time


In [ ]:
from datetime import timedelta

import pandas as pd

from snowflake.ml.model import custom_model
from snowflake.ml.model import model_signature
from snowflake.ml.registry import registry
from snowflake.ml.utils.connection_params import SnowflakeLoginOptions
from snowflake.snowpark import Session
from snowflake.snowpark import functions as F

from joblib import Parallel,delayed

In [ ]:
from snowflake.snowpark.version import VERSION
snowflake_environment = session.sql('select current_user(), current_version()').collect()
snowpark_version = VERSION

from snowflake.ml import version
mlversion = version.VERSION

# Current Environment Details
print('User                        : {}'.format(snowflake_environment[0][0]))
print('Role                        : {}'.format(session.get_current_role()))
print('Database                    : {}'.format(session.get_current_database()))
print('Schema                      : {}'.format(session.get_current_schema()))
print('Warehouse                   : {}'.format(session.get_current_warehouse()))
print('Snowflake version           : {}'.format(snowflake_environment[0][1]))
print('Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0],snowpark_version[1],snowpark_version[2]))
print('Snowflake ML version        : {}.{}.{}'.format(mlversion[0],mlversion[2],mlversion[4]))

In [ ]:
REGISTRY_DATABASE_NAME = "TPCDS_XGBOOST"
REGISTRY_SCHEMA_NAME = "DEMO"
reg = registry.Registry(session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME)

## Generate Data

In [ ]:
#Generate Series - Takes 2 minutes to run
#Only need to run this the first time
from statsforecast.utils import generate_series
#for length in [10_000, 100_000, 500_000, 1_000_000, 2_000_000]:
for length in [500_000]:
		print(f'length: {length}')
		series = generate_series(n_series=length, seed=1)

series

In [ ]:
# Prep data for training
df = pd.DataFrame(series)
train_df = df.reset_index()
train_df.columns = ['ID', 'DS', 'Y']
train_df

## Train Model Locally

you want to use pandas for initial local testing

In [ ]:
# Local Test for Exponential Smoothing
from statsmodels.tsa.holtwinters import ExponentialSmoothing
df = train_df[train_df['ID'] == 1]
df.set_index('DS', inplace=True)
model = ExponentialSmoothing(df['Y'], seasonal=None, trend='add', damped_trend=False)
fit = model.fit()
forecast = fit.forecast(steps=6)
forecast_df = pd.DataFrame({
                'DATE': forecast.index,
                'FORECAST': forecast.values
            })
forecast_df.head(6)

In [ ]:
## ES Model
class ForecastingModel(custom_model.CustomModel):
    # Use the same decorator as for methods with FUNCTION inference.
    @custom_model.partitioned_inference_api
    def predict(self, df:pd.DataFrame) -> pd.DataFrame:    #Please keep input and output here as pandas   
        ################## Replace below with your python code ######################################## 
        import pandas as pd
        from statsmodels.tsa.holtwinters import ExponentialSmoothing
        from datetime import datetime, timedelta
        df.set_index('DS', inplace=True)
        df = df.asfreq('D') 
        model = ExponentialSmoothing(df['Y'], seasonal=None, trend='add', damped_trend=False,freq='D')
        fit = model.fit()
        forecast = fit.forecast(steps=7)
        forecast_df = pd.DataFrame({
                        'DATE': forecast.index,
                        'FORECAST': forecast.values
                    })
        return forecast_df

In [ ]:
es_model = ForecastingModel()
local_predictions = es_model.predict(train_df[train_df['ID'] == 1])
#local_predictions = es_model.predict(train_df)
local_predictions

In [ ]:
# Function to parallelize
def parallel_predict(all_data, model):
    return model.predict(all_data)

# Assuming df1 is your complete dataset
all_groups = [group for _, group in train_df.groupby('ID')]

# Initialize the ForecastingModel
model = ForecastingModel()

# Parallel execution using Joblib
num_cores = -1  # Use all available cores
results = Parallel(n_jobs=num_cores)(delayed(parallel_predict)(all_data, model) for all_data in all_groups)

# Combine or process the results as needed

## Train Model in Snowflake

In [ ]:
options = {
    "function_type": "TABLE_FUNCTION",
}
mv = reg.log_model(
    es_model,
    model_name="es_forecast",
    version_name="v9",
    conda_dependencies=['pandas', 'statsmodels', 'snowflake-snowpark-python'],
    options=options,
    signatures={
        "predict": model_signature.ModelSignature(
            inputs=[
                model_signature.FeatureSpec(name="ID", dtype=model_signature.DataType.INT64),
                model_signature.FeatureSpec(name="DS", dtype=model_signature.DataType.TIMESTAMP_NTZ),
                model_signature.FeatureSpec(name="Y", dtype=model_signature.DataType.FLOAT),
            ],
            outputs=[
                model_signature.FeatureSpec(name="DSOUT", dtype=model_signature.DataType.TIMESTAMP_NTZ),
                model_signature.FeatureSpec(name="FORECAST", dtype=model_signature.DataType.FLOAT),
            ],
         )
     },
)

In [ ]:
reg_model = reg.get_model("es_forecast").version("v9")

In [ ]:
## running the pandas dataframe in a distributed way for training the models
result = reg_model.run(train_df, partition_column="ID")

In [ ]:
result.head(15)

In [ ]:
## Let's get a snowpark dataframe 
test_df = session.create_dataframe(train_df)
test_df.write.mode('overwrite').save_as_table('TPCDS_XGBOOST.DEMO.TEMPTS')
df2 = session.table('TPCDS_XGBOOST.DEMO.TEMPTS')
df2.show()

In [ ]:
##Run the model across the Snowflake DF. It should be faster than the pandas dataframe - especially at scale.
result = reg_model.run(df2, partition_column="ID",function_name="PREDICT").collect()

In [ ]:
snowpark_opt_wh = Warehouse(
  name="snowpark_opt_wh",
  warehouse_size="MEDIUM",
  warehouse_type = "SNOWPARK-OPTIMIZED",
  auto_suspend=600,
)
warehouses = root.warehouses["snowpark_opt_wh"]
warehouses.create_or_alter(snowpark_opt_wh)
session.use_warehouse("snowpark_opt_wh")

session.sql('alter session set USE_CACHED_RESULT = FALSE').collect()
session.sql('alter session set query_tag = "TS-LARGE-Chase" ').collect()
#session.sql('alter warehouse snowpark_opt_wh set max_concurrency_level = 1').collect()

print(session.get_current_warehouse())

In [72]:
lengths = [10_000, 50_000, 100_000, 500_000, 1_000_000,2_000_000]
#lengths = [5_000]

train_df = session.table('TPCDS_XGBOOST.DEMO.SERIES2M')
session.sql('ALTER TABLE TPCDS_XGBOOST.DEMO.SERIES2M CLUSTER BY (ID);').collect()


for length in lengths:
  print ("prepping data")
  df2 = train_df.filter((col("ID") >= 0) & (col("ID") <= (length-1)))
  print ("starting training")
  init = time()
  # Run the regression model
  result = reg_model.run(df2, partition_column="ID").collect()
  total_time = (time() - init) / 60
  print(f'n_series: {length} total time: {total_time} total rows: {filtered_df.count()}')

n_series: 2000000 total time: 33.24094546238582 total rows: 549884998
